# Interactive Sonification Mockup Notebook

panson = sonify everything
* pan --> from the greek παν: everything
* son --> sonify

TODO:
* serialize/deserialize sonifications (with parameters)
    * save to disk
* register callbacks
    * at initialization time
    * at each playback
    * at export time

## Sonification, DataPlayer, RTDataPlayer

In [ ]:
# import framework
import panson as ps # maybe it is easy to confuse with pd
import pandas as pd

import sc3nb as scn
scn.startup()

Create sonification objects by inheriting from the Sonification class (abstract base class). This object will be used in **both realtime and non-realtime** contexts.
* we could insert in the framework a little library of working sonifications (based on some sample data)

The sonification logic should be written just once and have both NRT and normal sonification available.

In [2]:
class MySonification(ps.Sonification):
    
    def initialize():
        pass
    
    # sonify the current row and return it in a bundle?
    def sonify(row):
        pass
    
    def end():
        pass

In [ ]:
son = MySonification(parameters)
son

When evaluated in the notebook, son should return a widget to control the parameters of the sonification. If we change the parameters, the playing sonification should update.

The parameters of the sonification can be changed:
* programmatically, e.g. son.amp = 0.3
* through the widget interface
    * it's probably better to use input fields rather than sliders (or maybe use both)

### Offline

In [ ]:
df = pd.read_csv('data.csv')
df.head()

The data player takes as an input the **sonification** to be used and the **dataframe** to be sonified.
* we may also allow the user to set or modify them after the creation of the object

In [ ]:
dp = ps.DataPlayer(son, df)
dp

When evaluated in the notebook, dp should return a widget to control the playback of the data. The interface would be ideally similar to widgets.Play, but with more options.
* we cannot use widgets.Play directly because its logic is not appropriate for our purposes

The data player should be able to specify a **constant playback rate** or to use **timestamp information** in the data.
* `DataPlayer(son, df, playback_rate=30)`
    * 30 data rows per second
    * there can't be any default for playback_rate that is meaningful with every data
        * when the argument is specified the DataPlayer will not consider any timestamp information
        * when the argument is not specified the DataPlayer will look for timestamp information in the data
            * `df['timestamp']` by default
            * `DataPlayer(son, df, timestamp_key='time')`
                * this way we can override the default timestamp lookup key
                

The DataPlayer object could also register **multiple sonification objects**, so that we could have more modular design, e.g. use separate sonifications for the smile and for the eyebrows and play them together.

In [ ]:
def update_video(df_row):
    # update videoframe
    pass

dp.add_callback('update_video', update_video)
# dp.del_callback('update_video')

To make it useful we have to allow the registration of callbacks also during initialization (and in other moments).

#### Real-time sonification

Control the data playback by using the **widget interface** or by using the **following methods**.

In [ ]:
dp.play()
# dp.play(rate=2)
# dp.play(rate=-1)

In [ ]:
dp.pause()

In [ ]:
dp.stop()

In [ ]:
dp.seek(frame_idx)

In [ ]:
dp.seektime(timestamp)

Plus other possible navigation functions...

The following method would record what is currently being played.
* NOTE: maybe its not a good idea to call the class DataPlayer if it allows recording

In [ ]:
dp.record_start('recording.wav')

In [ ]:
dp.record_stop()

What is the relationship between record_stop and the navigation methods? Should they be independent or not?
* pause
* stop

In [ ]:
# record for 10 second (the recording stops automatically)
dp.record_start('recording.wav', duration=10)

#### Non real-time sonification

We can render the NRT sonification using the following

In [ ]:
dp.export('sonification.wav')

Maybe export is not the best name:
* render
* nrt

### Online

Streamz (https://streamz.readthedocs.io/en/latest/core.html) seems to add only microsecond overhead to normal Python operations.
* there should not be any performance issues using it
* maybe we don't really need the library and we can write easier code without it

We could write a mainloop where read live data and input it into the RTDataPlayer, but this would block the main thread. This is a problem because we would want to interact programmatically with the RTDataPlayer later on.

It is better to encapsulate this in a method and run it in a separate thread.

The acquisition of live feature is too dependent on the context to code it all in advance. The mainloop code should be specified by the user. We could pass a function at object creation time, but it's clearer if we define RTDataPlayer as an abstract base class and we instantiate custom subclasses of it. E.g. in NamedPipeDataPlayer the data player would read data from a specified named pipe

In [ ]:
class NamedPipeDataPlayer(ps.RTDataPlayer):
    
    def data_loop(self, fifo_path):
        # open a named pipe and parse the data in the expected format        
        with open(fifo_path, 'r') as fifo:
            # the reader attempts to execute fifo.readline() (which is blocking if there are no lines)
            reader = csv.reader(fifo)

            # the loop ends when the pipe is closed from the writing side
            for row in reader:
                # input data the read row in the streamz Stream object
                self.source.emit(row)

rtdp = NamedPipeDataPlayer(son)

In [ ]:
# start the flow of data
rtdp.listen()

In [ ]:
# record live sounds as with the DataPlayer
rtdp.record_start('recording.wav')

Log real-time data with the following functions.

In [ ]:
rtdp.log_start()

log_stop can maybe return a DataPlayer object to be able to perform the playback of the logged data immediately.

In [ ]:
rtdp.log_stop()

### Problems
* It is not clear which are the responsabilities of the objects, e.g. DataPlayers allow recording
    * change its name?
* How to write sonification logic only once?
    * row sonification function could return a bundle, so that the routing part (to the server or to a score file through NRT) would be handled by the framework
* RTDataPlayer can be hard to understand, even if the method should be flexible
* Jupyter notebook widgets are able to support all these operations?

## MVC based
* Model: SonificationModel (OnlineDataModel, OfflineDataModel)
* View: OnlineView, OfflineView
* Controller: OfflineController, OnlineController

This seems to be a standard and easily understandable design, but I don't know if it is a good idea to apply it in a jupyter notebook environment:
* the programmatic interaction could become more complex
    * should we also force the programmatic interaction through the controller?
    * the MVC is usually used in different contexts, where the only interaction happens through the GUI
* in a jupyter notebook we could have various sonifications rendered through widgets and plug only one at a time in the DataPlayer
    * usually the view is conceived as a monolithic entity, which means that we should render the sonification widget together with the playback widget. This would make the everything less pluginable and flexible, e.g. we have to re-render all the view if we change the sonification
        * is it actually a problem?
* A MVC architecture applied in a context where it does not fit well (i.e. where the user of the framework has other expectations) could generate a lot of confusion.
    * maybe it would be better to adopt a completely different structure

## Sonification, DataProcessor, RTDataProcessor

This solution is kind of similar to Thomas' one.

* Sonification as in the first example
* DataProcessor - coordinates the following elements:
    * Sonification
    * OfflineData (does not necessairly have to be an object)
    * DataPlayer
* RTDataProcessor - coordinates the following elements:
    * Sonification
    * OnlineData (does not necessairly have to be an object)
        * we can leverage streamz and use the emit method on a source (attribute)

In the RTDataProcessor we don't need any RTDataPlayer, because there is no navigation and playback to perform. The main tasks of the RTDataProcessor are:
* call the sonification on the current element obtained from the stream
* log (save) the stream of data
* does it makes sense to record the sounds produced by the sonification directly? Surely there must be a way to synchronize it with the logged data... the mechanism should be analogous to the one of DataProcessor.

RTDataProcessor can also be used as a builder for DataProcessor (after recording).

How can we handle streams of video or audio data?

The framework should also contain some objects to simplify the building of the widget interface.

## Thomas' idea

In [ ]:
import isfw as sf # import the "interactive sonification framework" package as sf 
import sc3nb as scn
scn.startup()

In [ ]:
rts = sf.RTStream() # create a rt stream for sensor data. This needs to be worked out... 

dp = sf.DataProcessor(source=rts) # create a data processor with data source connected to a rtstream 

dp.son = sf.PercussiveSon() # this needs to be worked out...., perhaps allow to pass an array?

In [ ]:
# configure logging
dp.logfile = "" # default None, but if set, logging goes into file, 
                # if object is a DataFrame, log lines are appended
dp.logging = True # default is False, True immediately could immediately start logging

In [ ]:
# run the processor
dp.run() # this would automatically call self.son.initialize()
         # and then for every incoming sensor data vector self.son.update(dp, dp.current_data)

now we can enjoy the sonification and perhaps tweak it a bit like this

In [ ]:
dp.son.level += 6 # add 6 dB global volume

In [ ]:
dp.son.mute() # to have silence for discussing with a colleague

In [ ]:
dp.son.unmute() # to continue listening

In [ ]:
dp.running   # should return True or False

In [ ]:
dp.source = None # disconnect the source, e.g. because we want to continue with recorded data

In [ ]:
# dp.source = DataPlayer()
pl = sf.DataPlayer(dp.data.copy()) # switch to just logged data from previous rt interaction
dp.source = pl
pl

would give the repr which states something like:<br>
```150 frames, avg fps 21.5, dim=18``` 

In [ ]:
pl.loop(from=80, to=-1) # configure player to loop segment from frame 80 to end
pl.cue(frame=80)
pl.start() # now we enjoy the sonification in a loop

In [ ]:
# we would like to attend to this in more detail and slow down by a factor of 2
pl.rate = 0.5 # the player stretches times between rows by a 1/rate and so the sonification slows down

In [ ]:
# now we can work on some sonification parameters, let's for instance shorten the percussive sonification

In [ ]:
dp.son.anyproperty_available_in_class *= 0.5 # in this case we assume we have a property for event duration...

In [ ]:
# more convenient: create a widget to control a property
wdg_1 = sf.parameter_widget(dp.son.anyproperty_available_in_class, (0.1, 1, 100))
# in result of which we get a slider widget from 0.1 to 1 in 100 steps 
# so that we can control the parameter in realtime

We find that we need to see the video in parallel, so here we need to synchronize a video playback component with the data

In [ ]:
# for coding we stop temporarily the dp
dp.pause() 
# actualy we could let it run but just stop new data from coming int
dp.resume(); pl.pause()

In [ ]:
# let's interactively load the data
dp.vidview = sf.VideoViewer("MyVideo.mp4", mode="memory") # loading the full video into memory
dp.vidview

# current doubt: perhaps vidview should not be part of dp but connected to pl? makes more sense...
# such are issues to be thought about...

this could output the __repr__(), e.g.:<br>
```Video: 18s at 30fps = 540 frames of res 800 x 640 x 3 (RGB)```

In [ ]:
dp.vidview.display() # a window pops up showing the first frame

In [ ]:
sf.parameter_widget(dp.vidview.currentframe, (0, dp.vidview.nrframes, 1))
# a slider pops up which allows us to browse the video...

In [ ]:
# now lets play the video together with the sonification.
# as it is already registered as vidview, it would automatically receive the frame number from the pd, so
pl.resume() # will present